In [6]:
import json
import os

import requests
from langchain.tools import tool
from dotenv import load_dotenv
load_dotenv()



True

In [13]:
def search_internet(query):
    url = "https://google.serper.dev/search"
    payload = json.dumps({"q": query})
    headers = {
        'X-API-KEY': os.environ['SERPER_API_KEY'],
        'content-type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    return response

In [14]:
response = search_internet("BMW")

In [15]:
response.json()

{'searchParameters': {'q': 'BMW', 'type': 'search', 'engine': 'google'},
 'knowledgeGraph': {'title': 'Bayerische Motoren Werke AG',
  'type': 'Luxury vehicles company',
  'imageUrl': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSm7U2BJDwRbg0bfghu9V4ttun74eUtlDOSt8KtzSTK7PomdlTswB5wnA&s=0',
  'description': 'Bayerische Motoren Werke AG, commonly abbreviated to BMW, is a German multinational manufacturer of luxury vehicles and motorcycles headquartered in Munich, Bavaria, Germany.',
  'descriptionSource': 'Wikipedia',
  'descriptionLink': 'https://en.wikipedia.org/wiki/BMW',
  'attributes': {'Owners': 'Susanne Klatten and Stefan Quandt',
   'Founders': 'Karl Rapp, Gustav Otto, Camillo Castiglioni, and more',
   'Subsidiaries': 'BMW Motorrad, BMW M, Alpina, and more',
   'Customer service': '1 (800) 831-1117',
   'Revenue': '142.6\xa0billion EUR (2022)'}},
 'organic': [{'title': 'BMW USA: Luxury SUVs, Sedans, Coupes, Convertibles & Crossovers',
   'link': 'https://www.bmwusa.co

In [16]:
response.json()['organic']

[{'title': 'BMW USA: Luxury SUVs, Sedans, Coupes, Convertibles & Crossovers',
  'link': 'https://www.bmwusa.com/',
  'snippet': 'Explore models, build your own, and find local inventory from a nearby BMW Center. Experience the performance, luxury, and innovation of the Ultimate ...',
  'sitelinks': [{'title': 'Build Your Own',
    'link': 'https://www.bmwusa.com/build-your-own.html'},
   {'title': 'All BMW Models', 'link': 'https://www.bmwusa.com/all-bmws.html'},
   {'title': 'Shop Pre-Owned Inventory',
    'link': 'https://www.bmwusa.com/certified-preowned-search.html'},
   {'title': 'Shop New Inventory',
    'link': 'https://www.bmwusa.com/inventory.html'},
   {'title': 'Lease & Finance Offers',
    'link': 'https://www.bmwusa.com/special-offers.html'}],
  'position': 1},
 {'title': 'Home | BMW Motorrad',
  'link': 'https://www.bmwmotorcycles.com/en/home.html',
  'snippet': 'Manage your BMW ID for all BMW Motorrad, BMW and MINI offers. My BMW Motorrad ... The BMW Motorrad Concept Lin

In [21]:
top_result_to_return = 4
def search_news(query):
    url = "https://google.serper.dev/news"
    payload = json.dumps({"q": query})
    headers = {
        'X-API-KEY': os.environ['SERPER_API_KEY'],
        'content-type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    results = response.json()['news']
    string = []
    for result in results[:top_result_to_return]:
      try:
        string.append('\n'.join([
            f"Title: {result['title']}", f"Link: {result['link']}",
            f"Snippet: {result['snippet']}", "\n-----------------"
        ]))
      except KeyError:
        next

    return '\n'.join(string)

In [22]:
news_response = search_news("BMW")

In [23]:
news_response

"Title: The 2024 BMW M3 CS Is Stupid Fast\nLink: https://www.motor1.com/reviews/725272/2024-bmw-m3-cs-review-track-test/\nSnippet: We drive the 2024 BMW M3 CS and discover just how quick and capable it can be at Lime Rock Park, Connecticut's historic race track.\n\n-----------------"

In [24]:
from langchain.tools.yahoo_finance_news import YahooFinanceNewsTool


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [36]:
t = YahooFinanceNewsTool(query="GOOG")

In [34]:
ynews

<Response [404]>

In [37]:
from sec_api import QueryApi
stock = "GOOG"
if stock:
    queryApi = QueryApi(api_key=os.environ['SEC_API_API_KEY'])
    query = {
      "query": {
        "query_string": {
          "query": f"ticker:{stock} AND formType:\"10-Q\""
        }
      },
      "from": "0",
      "size": "1",
      "sort": [{ "filedAt": { "order": "desc" }}]
    }

    fillings = queryApi.get_filings(query)['filings']
    if len(fillings) == 0:
      print("Sorry, I couldn't find any filling for this stock, check if the ticker is correct.")
    link = fillings[0]['linkToFilingDetails']

In [38]:
fillings

[{'ticker': 'GOOG',
  'formType': '10-Q',
  'accessionNo': '0001652044-24-000053',
  'cik': '1652044',
  'companyNameLong': 'Alphabet Inc. (Filer)',
  'companyName': 'Alphabet Inc.',
  'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/1652044/000165204424000053/goog-20240331.htm',
  'description': 'Form 10-Q - Quarterly report [Sections 13 or 15(d)]',
  'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1652044/000165204424000053/0001652044-24-000053.txt',
  'filedAt': '2024-04-25T20:22:11-04:00',
  'documentFormatFiles': [{'sequence': '1',
    'size': '1493403',
    'documentUrl': 'https://www.sec.gov/ix?doc=/Archives/edgar/data/1652044/000165204424000053/goog-20240331.htm',
    'description': '10-Q',
    'type': '10-Q'},
   {'sequence': '2',
    'size': '51747',
    'documentUrl': 'https://www.sec.gov/Archives/edgar/data/1652044/000165204424000053/googexhibit1001q12024.htm',
    'description': 'EX-10.01',
    'type': 'EX-10.01'},
   {'sequence': '3',
    'size': '788

In [40]:
def __download_form_html(url):
    headers = {
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
      'Accept-Encoding': 'gzip, deflate, br',
      'Accept-Language': 'en-US,en;q=0.9,pt-BR;q=0.8,pt;q=0.7',
      'Cache-Control': 'max-age=0',
      'Dnt': '1',
      'Sec-Ch-Ua': '"Not_A Brand";v="8", "Chromium";v="120"',
      'Sec-Ch-Ua-Mobile': '?0',
      'Sec-Ch-Ua-Platform': '"macOS"',
      'Sec-Fetch-Dest': 'document',
      'Sec-Fetch-Mode': 'navigate',
      'Sec-Fetch-Site': 'none',
      'Sec-Fetch-User': '?1',
      'Upgrade-Insecure-Requests': '1',
      'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    return response.text

In [41]:
from langchain.tools import tool
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from unstructured.partition.html import partition_html

def __embedding_search(url, ask):
    text = __download_form_html(url)
    elements = partition_html(text=text)
    content = "\n".join([str(el) for el in elements])
    text_splitter = CharacterTextSplitter(
        separator = "\n",
        chunk_size = 1000,
        chunk_overlap  = 150,
        length_function = len,
        is_separator_regex = False,
    )
    docs = text_splitter.create_documents([content])
    retriever = FAISS.from_documents(
      docs, OpenAIEmbeddings()
    ).as_retriever()
    answers = retriever.get_relevant_documents(ask, top_k=4)
    answers = "\n\n".join([a.page_content for a in answers])
    return answers

In [43]:
link = fillings[0]['linkToFilingDetails']
ask = "what was last year's revenue"
answer = __embedding_search(link, ask)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/deepakmishra/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/deepakmishra/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
Created a chunk of size 1993, which is longer than the specified 1000
Created a chunk of size 1863, which is longer than the specified 1000
Created a chunk of size 1813, which is longer than the specified 1000
Created a chunk of size 1146, which is longer than the specified 1000
Created a chunk of size 1141, which is longer than the specified 1000
Created a chunk of size 1014, which is longer than the specified 1000
Created a chunk of size 1146, which is longer than the specified 1000
Created a chunk of size 1147, which is longer than the specified 1000
Created a chunk of size 1244, which is longer than the specified 1000
Created a chunk of size 1323, which is longer than the spec

In [44]:
answer

'Total constant currency revenues, which exclude the effect of hedging, increased 16% year over year.\nCost of revenues was $33.7 billion, an increase of 10% year over year, primarily driven by increases in TAC, content acquisition costs, and depreciation expense. These increases were partially offset by decreases in compensation expenses, largely the result of a reduction in employee severance and related charges, and charges related to our office space optimization efforts.\nOperating expenses were $21.4 billion, a decrease of 2% year over year, primarily driven by decreases in charges related to our office space optimization efforts, compensation expenses, largely the result of a reduction in employee severance and related charges, and charges related to legal matters. These decreases were partially offset by an increase in depreciation expense.\n32\nOther Information\n\nAs of March\xa031, 2024, we had 180,895 employees.\nFinancial Results\nRevenues\nThe following table presents rev